### <center> PREFERENCES AS BINARY RELATIONS </center>
#### <center> MASTER BDMA - DECISION MODELING </center>
##### <center> Authors: Dilbar Isakova, MD Kamrul Islam </center>
##### <center> Professor: Brice Mayag </center>
###### <center> CentraleSupeléc, Fall, 2024 </center>

In [37]:
import pandas as pd
import numpy as np
from math import sqrt
import random

#### DataFrame with the movie critics' ratings

In [38]:
data = {
    'Critic': ['Lisa Rose', 'Gene Seymour', 'Michael Phillips', 'Claudia Puig', 'Mick Lasalle', 'Jack Matthews', 'Toby', 'Anne'],
    'Lady': [2.5, 3.0, 2.5, None, 3.0, 3.0, None, 1.5],
    'Snakes': [3.5, 3.5, 3.0, 3.5, 4.0, 4.0, 4.5, None],
    'Luck': [3.0, 1.5, None, 3.0, 2.0, None, None, 4.0],
    'Superman': [3.5, 5.0, 3.5, 4.0, 3.0, 5.0, 4.0, None],
    'Dupree': [2.5, 3.5, None, 2.5, 2.0, 3.5, 1.0, 2.0],
    'Night': [3.0, 3.0, 4.0, 4.5, 3.0, 3.0, None, None]
}
df = pd.DataFrame(data)

excel_file_path = 'movie_ratings.xlsx'
df.to_excel(excel_file_path, index=False)


## Question 01

In [39]:
df = pd.read_excel(excel_file_path)
df.set_index('Critic', inplace=True)

# Transpose and convert to dictionary
critiques = df.transpose().to_dict()

# Check Lisa Rose's ratings
print("Critiques['Lisa Rose']:", critiques['Lisa Rose'])

Critiques['Lisa Rose']: {'Lady': 2.5, 'Snakes': 3.5, 'Luck': 3.0, 'Superman': 3.5, 'Dupree': 2.5, 'Night': 3.0}


## Question 02(a)

In [40]:
def sim_distanceManhattan(person1, person2):
    shared_movies = [
        movie for movie in person1
        if movie in person2 and not pd.isna(person1[movie]) and not pd.isna(person2[movie])
    ]
    if not shared_movies:
        return float('inf') 
    # Calculate Manhattan distance
    ratings1 = np.array([person1[movie] for movie in shared_movies])
    ratings2 = np.array([person2[movie] for movie in shared_movies])
    distance = np.sum(np.abs(ratings1 - ratings2))
    return distance

def sim_distanceEuclidienne(person1, person2):
    shared_movies = [
        movie for movie in person1
        if movie in person2 and not pd.isna(person1[movie]) and not pd.isna(person2[movie])
    ]
    if not shared_movies:
        return float('inf')
    # Calculate Euclidean distance
    ratings1 = np.array([person1[movie] for movie in shared_movies])
    ratings2 = np.array([person2[movie] for movie in shared_movies])
    distance = np.sqrt(np.sum((ratings1 - ratings2) ** 2))
    return distance

In [41]:
manhattan_distance = sim_distanceManhattan(critiques['Lisa Rose'], critiques['Gene Seymour'])
euclidean_distance = sim_distanceEuclidienne(critiques['Lisa Rose'], critiques['Gene Seymour'])

print("Manhattan Distance between Lisa Rose and Gene Seymour:", manhattan_distance)
print("Euclidean Distance between Lisa Rose and Gene Seymour:", euclidean_distance)

Manhattan Distance between Lisa Rose and Gene Seymour: 4.5
Euclidean Distance between Lisa Rose and Gene Seymour: 2.3979157616563596


## Question 02(b)

In [42]:
def computeNearestNeighbor(nouveauCritique, critiques):
    distances = []
    for critique in critiques:
        if critique != nouveauCritique:
            distance = sim_distanceManhattan(critiques[critique], critiques[nouveauCritique])
            distances.append((distance, critique))
    distances.sort()
    return distances

def recommendNearestNeighbor(nouveauCritique, critiques):

    nearest = computeNearestNeighbor(nouveauCritique, critiques)
    for distance, neighbor in nearest:
        neighbor_ratings = critiques[neighbor]
        user_ratings = critiques[nouveauCritique]
        recommendations = []
        for movie in neighbor_ratings:
            if (movie not in user_ratings or pd.isna(user_ratings[movie])) and not pd.isna(neighbor_ratings[movie]):
                recommendations.append((movie, neighbor_ratings[movie]))
        if recommendations:
            return recommendations
    return []

In [43]:
print("Nearest neighbors for 'Lisa Rose':")
print(computeNearestNeighbor('Lisa Rose', critiques))
print("\nRecommendations for 'Lisa Rose':")
print(recommendNearestNeighbor('Lisa Rose', critiques))

print("\nNearest neighbors for 'Toby':")
print(computeNearestNeighbor('Toby', critiques))
print("\nRecommendations for 'Toby':")
print(recommendNearestNeighbor('Toby', critiques))

Nearest neighbors for 'Lisa Rose':
[(1.5, 'Michael Phillips'), (2.0, 'Claudia Puig'), (2.5, 'Anne'), (3.0, 'Mick Lasalle'), (3.0, 'Toby'), (3.5, 'Jack Matthews'), (4.5, 'Gene Seymour')]

Recommendations for 'Lisa Rose':
[]

Nearest neighbors for 'Toby':
[(1.0, 'Anne'), (2.0, 'Michael Phillips'), (2.5, 'Claudia Puig'), (2.5, 'Mick Lasalle'), (3.0, 'Lisa Rose'), (4.0, 'Jack Matthews'), (4.5, 'Gene Seymour')]

Recommendations for 'Toby':
[('Lady', 1.5), ('Luck', 4.0)]


## Question 2(c)


In [44]:
# Bestrecommend function
def Bestrecommend(nouveauCritique, critiques, movies_to_consider, distance_metric):
    totals = {}
    sim_sums = {}
    for movie in movies_to_consider:
        totals[movie] = 0.0
        sim_sums[movie] = 0.0

    for critic in critiques:
        if critic == nouveauCritique:
            continue
        distance = distance_metric(critiques[critic], critiques[nouveauCritique])
        if distance == float('inf'):
            continue 
        weight = 1 / (1 + distance)
        if weight <= 0:
            continue 
        for movie in movies_to_consider:
            # Only consider movies that critic has rated and Anne hasn't
            if movie in critiques[critic] and not pd.isna(critiques[critic][movie]):
                if movie not in critiques[nouveauCritique] or pd.isna(critiques[nouveauCritique][movie]):
                    totals[movie] += critiques[critic][movie] * weight
                    sim_sums[movie] += weight

    rankings = []
    for movie in movies_to_consider:
        if sim_sums[movie] != 0:
            score = totals[movie] / sim_sums[movie]
            rankings.append((round(score, 2), movie))
    rankings.sort(reverse=True)
    return rankings


# BestrecommendwithExp function
def BestrecommendwithExp(nouveauCritique, critiques, movies_to_consider, distance_metric):
    totals = {}
    sim_sums = {}
    for movie in movies_to_consider:
        totals[movie] = 0.0
        sim_sums[movie] = 0.0

    for critic in critiques:
        if critic == nouveauCritique:
            continue
        distance = distance_metric(critiques[critic], critiques[nouveauCritique])
        if distance == float('inf'):
            continue 
        weight = np.exp(-distance)
        if weight <= 0:
            continue 
        for movie in movies_to_consider:
            # Only consider movies that critic has rated and Anne hasn't
            if movie in critiques[critic] and not pd.isna(critiques[critic][movie]):
                if movie not in critiques[nouveauCritique] or pd.isna(critiques[nouveauCritique][movie]):
                    totals[movie] += critiques[critic][movie] * weight
                    sim_sums[movie] += weight

    rankings = []
    for movie in movies_to_consider:
        if sim_sums[movie] != 0:
            score = totals[movie] / sim_sums[movie]
            rankings.append((score, movie))
    rankings.sort(reverse=True)
    return rankings

In [45]:
# Movies to consider
movies_to_consider = ['Snakes', 'Superman', 'Night']

print("Best recommendations for Anne using Manhattan distance:")
manhattan_recommendations = Bestrecommend('Anne', critiques, movies_to_consider, sim_distanceManhattan)
for score, movie in manhattan_recommendations:
    print(f"Movie: {movie}, Predicted Rating: {score:.2f}")

print("\nBest recommendations for Anne using Euclidean distance:")
euclidean_recommendations = Bestrecommend('Anne', critiques, movies_to_consider, sim_distanceEuclidienne)
for score, movie in euclidean_recommendations:
    print(f"Movie: {movie}, Predicted Rating: {score:.2f}")

print("\nBest recommendations for Anne using Manhattan distance with exponential weights:")
exp_manhattan_recommendations = BestrecommendwithExp('Anne', critiques, movies_to_consider, sim_distanceManhattan)
for score, movie in exp_manhattan_recommendations:
    print(f"Movie: {movie}, Predicted Rating: {score:.2f}")

print("\nBest recommendations for Anne using Euclidean distance with exponential weights:")
exp_euclidean_recommendations = BestrecommendwithExp('Anne', critiques, movies_to_consider, sim_distanceEuclidienne)
for score, movie in exp_euclidean_recommendations:
    print(f"Movie: {movie}, Predicted Rating: {score:.2f}")

Best recommendations for Anne using Manhattan distance:
Movie: Superman, Predicted Rating: 3.91
Movie: Snakes, Predicted Rating: 3.71
Movie: Night, Predicted Rating: 3.61

Best recommendations for Anne using Euclidean distance:
Movie: Superman, Predicted Rating: 3.93
Movie: Snakes, Predicted Rating: 3.70
Movie: Night, Predicted Rating: 3.55

Best recommendations for Anne using Manhattan distance with exponential weights:
Movie: Night, Predicted Rating: 3.93
Movie: Superman, Predicted Rating: 3.82
Movie: Snakes, Predicted Rating: 3.70

Best recommendations for Anne using Euclidean distance with exponential weights:
Movie: Superman, Predicted Rating: 3.86
Movie: Night, Predicted Rating: 3.74
Movie: Snakes, Predicted Rating: 3.69


## Question 2(d)

In [46]:
# Pearson correlation function
def pearson(person1, person2):
    sum_xy = 0
    sum_x = 0
    sum_y = 0
    sum_x2 = 0
    sum_y2 = 0
    n = 0
    for key in person1:
        if key in person2:
            x = person1[key]
            y = person2[key]
            if not pd.isna(x) and not pd.isna(y):
                n += 1
                sum_xy += x * y
                sum_x += x
                sum_y += y
                sum_x2 += x**2
                sum_y2 += y**2
    if n == 0:
        return 0  # No ratings in common
    denominator = sqrt(sum_x2 - (sum_x**2) / n) * sqrt(sum_y2 - (sum_y**2) / n)
    if denominator == 0:
        return 0
    else:
        numerator = sum_xy - (sum_x * sum_y) / n
        return numerator / denominator
    
# PearsonRecommend function
def PearsonRecommend(nouveauCritique, critiques, movies_to_consider):
    totals = {}
    sim_sums = {}
    for movie in movies_to_consider:
        totals[movie] = 0.0
        sim_sums[movie] = 0.0

    for critic in critiques:
        if critic == nouveauCritique:
            continue
        similarity = pearson(critiques[critic], critiques[nouveauCritique])
        if similarity == 0:
            continue  # No similarity
        for movie in movies_to_consider:
            if movie in critiques[critic] and not pd.isna(critiques[critic][movie]):
                if movie not in critiques[nouveauCritique] or pd.isna(critiques[nouveauCritique][movie]):
                    totals[movie] += critiques[critic][movie] * similarity
                    sim_sums[movie] += similarity

    rankings = []
    for movie in movies_to_consider:
        if sim_sums[movie] != 0:
            score = totals[movie] / sim_sums[movie]
            rankings.append((score, movie))
    rankings.sort(reverse=True)
    return rankings

In [47]:
# Movies to consider
movies_to_consider = ['Snakes', 'Superman', 'Night']

# Testing PearsonRecommend
print("Best recommendations for Anne using Pearson correlation coefficient:")
pearson_recommendations = PearsonRecommend('Anne', critiques, movies_to_consider)
for score, movie in pearson_recommendations:
    print(f"Movie: {movie}, Predicted Rating: {score:.2f}")

Best recommendations for Anne using Pearson correlation coefficient:
Movie: Superman, Predicted Rating: 4.18
Movie: Night, Predicted Rating: 4.06
Movie: Snakes, Predicted Rating: 3.62


## Question 2(e)

In [48]:
def cosine_similarity(person1, person2):
    sum_xy = 0
    sum_x2 = 0
    sum_y2 = 0
    for key in person1:
        if key in person2:
            x = person1[key]
            y = person2[key]
            if not pd.isna(x) and not pd.isna(y):
                sum_xy += x * y
                sum_x2 += x ** 2
                sum_y2 += y ** 2
    if sum_x2 == 0 or sum_y2 == 0:
        return 0 
    return sum_xy / (sqrt(sum_x2) * sqrt(sum_y2))

def CosineRecommend(nouveauCritique, critiques, movies_to_consider):
    totals = {}
    sim_sums = {}
    for movie in movies_to_consider:
        totals[movie] = 0.0
        sim_sums[movie] = 0.0

    for critic in critiques:
        if critic == nouveauCritique:
            continue
        similarity = cosine_similarity(critiques[critic], critiques[nouveauCritique])
        if similarity <= 0:
            continue 
        for movie in movies_to_consider:
            if movie in critiques[critic] and not pd.isna(critiques[critic][movie]):
                if movie not in critiques[nouveauCritique] or pd.isna(critiques[nouveauCritique][movie]):
                    totals[movie] += critiques[critic][movie] * similarity
                    sim_sums[movie] += similarity

    rankings = []
    for movie in movies_to_consider:
        if sim_sums[movie] != 0:
            score = totals[movie] / sim_sums[movie]
            rankings.append((round(score, 2), movie)) 
    rankings.sort(reverse=True)
    return rankings

In [49]:
movies_to_consider = ['Snakes', 'Superman', 'Night']

print("Best recommendations for Anne using Cosine similarity:")
cosine_recommendations = CosineRecommend('Anne', critiques, movies_to_consider)
for score, movie in cosine_recommendations:
    print(f"Movie: {movie}, Predicted Rating: {score:.2f}")

Best recommendations for Anne using Cosine similarity:
Movie: Superman, Predicted Rating: 3.99
Movie: Snakes, Predicted Rating: 3.72
Movie: Night, Predicted Rating: 3.44


## Question 03

In [50]:
data = {
    'Critic': ['Angelica', 'Bill', 'Chan', 'Dan', 'Hailey', 'Jordyn', 'Sam', 'Veronica'],
    'Blues Traveler': [3.5, 2, 5, 3, None, None, 5, 3],
    'Broken Bells': [2, 3.5, 1, 4, 4, 4.5, 2, None],
    'Deadmau5': [None, 4, 1, 4.5, 1, 4, None, None],
    'Norah Jones': [4.5, None, 3, None, 4, 5, 3, 5],
    'Phoenix': [5, 2, 5, 3, None, 5, 5, 4],
    'Slightly Stoopid': [1.5, 3.5, 1, 4.5, None, 4.5, 4, 2.5],
    'The Strokes': [2.5, None, None, 4, 4, 4, 5, 3],
    'Vampire Weekend': [2, 3, None, 2, 1, 4, None, None]
}

df = pd.DataFrame(data)
df.set_index('Critic', inplace=True)

critiques = df.transpose().to_dict()

In [51]:
# Songs to consider
songs_to_consider = ['Blues Traveler', 'Broken Bells', 'Deadmau5', 'Norah Jones', 'Phoenix', 'Slightly Stoopid', 'The Strokes', 'Vampire Weekend']
# Test results for Veronica and Hailey
results = {
    'Veronica': {
        'Manhattan': Bestrecommend('Veronica', critiques, songs_to_consider, sim_distanceManhattan),
        'Euclidean': Bestrecommend('Veronica', critiques, songs_to_consider, sim_distanceEuclidienne),
        'Pearson': Bestrecommend('Veronica', critiques, songs_to_consider, pearson),
        'Cosine': CosineRecommend('Veronica', critiques, songs_to_consider)
    },
    'Hailey': {
        'Manhattan': Bestrecommend('Hailey', critiques, songs_to_consider, sim_distanceManhattan),
        'Euclidean': Bestrecommend('Hailey', critiques, songs_to_consider, sim_distanceEuclidienne),
        'Pearson': Bestrecommend('Hailey', critiques, songs_to_consider, pearson),
        'Cosine': CosineRecommend('Hailey', critiques, songs_to_consider)
    }
}

# Print results with explanations
for person, recommendations in results.items():
    print(f"Recommendations for {person}:\n")
    for method, recs in recommendations.items():
        print(f"Using {method} similarity:")
        for score, song in recs:
            print(f" - {song}: Predicted Rating {score}")
        print()
    print("=" * 50 + "\n")

Recommendations for Veronica:

Using Manhattan similarity:
 - Broken Bells: Predicted Rating 3.24
 - Deadmau5: Predicted Rating 2.78
 - Vampire Weekend: Predicted Rating 2.23

Using Euclidean similarity:
 - Broken Bells: Predicted Rating 3.12
 - Deadmau5: Predicted Rating 2.82
 - Vampire Weekend: Predicted Rating 2.27

Using Pearson similarity:
 - Deadmau5: Predicted Rating 3.68
 - Broken Bells: Predicted Rating 3.27
 - Vampire Weekend: Predicted Rating 2.41

Using Cosine similarity:
 - Broken Bells: Predicted Rating 3.02
 - Deadmau5: Predicted Rating 2.91
 - Vampire Weekend: Predicted Rating 2.4


Recommendations for Hailey:

Using Manhattan similarity:
 - Phoenix: Predicted Rating 4.14
 - Blues Traveler: Predicted Rating 3.59
 - Slightly Stoopid: Predicted Rating 2.93

Using Euclidean similarity:
 - Phoenix: Predicted Rating 4.18
 - Blues Traveler: Predicted Rating 3.6
 - Slightly Stoopid: Predicted Rating 2.95

Using Pearson similarity:
 - Phoenix: Predicted Rating 4.05
 - Blues Tra

## Question 04

In [52]:
np.random.seed(42)
num_critics = 15
num_movies = 20
ratings = np.random.uniform(1, 5, size=(num_critics, num_movies))

# Introduce missing values (30%-50% missing data)
def apply_missing_data(matrix, missing_ratio=0.4):
    total_cells = matrix.size
    num_missing = int(total_cells * missing_ratio)
    indices = [(i, j) for i in range(matrix.shape[0]) for j in range(matrix.shape[1])]
    missing_indices = random.sample(indices, num_missing)
    for i, j in missing_indices:
        matrix[i, j] = np.nan
    return matrix

ratings = apply_missing_data(ratings.copy(), missing_ratio=0.4)

critics = [f"Critic_{i+1}" for i in range(num_critics)]
movies = [f"Movie_{j+1}" for j in range(num_movies)]
ratings_df = pd.DataFrame(ratings, index=critics, columns=movies)

# Ensure the target critic has not seen at least half of the movies
ratings_df.iloc[0, :10] = np.nan  # Use .iloc for positional indexing

In [53]:
def check_missing_data_proportion(df, min_percentage=0.3, max_percentage=0.5):
    total_cells = df.size
    missing_cells = df.isna().sum().sum()
    missing_percentage = missing_cells / total_cells
    return min_percentage <= missing_percentage <= max_percentage

# Test the function
if check_missing_data_proportion(ratings_df):
    print("The missing data condition (30% - 50%) is satisfied.")
else:
    print("The missing data condition is NOT satisfied.")

The missing data condition (30% - 50%) is satisfied.


In [54]:
# Chebyshev Distance
def sim_chebyshev(person1, person2):
    shared_movies = [
        movie for movie in person1
        if movie in person2 and not pd.isna(person1[movie]) and not pd.isna(person2[movie])
    ]
    if not shared_movies:
        return float('inf')  # No ratings in common
    ratings1 = np.array([person1[movie] for movie in shared_movies])
    ratings2 = np.array([person2[movie] for movie in shared_movies])
    distance = np.max(np.abs(ratings1 - ratings2))
    return distance

# Minkowski Distance (parameterized by p, e.g., p=3)
def sim_minkowski(person1, person2, p=3):
    shared_movies = [
        movie for movie in person1
        if movie in person2 and not pd.isna(person1[movie]) and not pd.isna(person2[movie])
    ]
    if not shared_movies:
        return float('inf')  # No ratings in common
    ratings1 = np.array([person1[movie] for movie in shared_movies])
    ratings2 = np.array([person2[movie] for movie in shared_movies])
    distance = np.sum(np.abs(ratings1 - ratings2) ** p) ** (1 / p)
    return distance

In [55]:
# Define the target critic
target_critic = 'Critic_2'

# Define the list of movies to consider for recommendations (all movies in the DataFrame)
movies_to_consider = ratings_df.columns.tolist()

# Define similarity measures with the appropriate functions
similarity_measures = {
    "Manhattan": sim_distanceManhattan,
    "Euclidean": sim_distanceEuclidienne,
    "Pearson": pearson,
    "Cosine": cosine_similarity,
    "Chebyshev": sim_chebyshev,
    "Minkowski": lambda p1, p2: sim_minkowski(p1, p2, p=3)
}

# Generate recommendations for each similarity measure
recommendations = {}
for name, func in similarity_measures.items():
    recommendations[name] = Bestrecommend(target_critic, ratings_df.to_dict(orient="index"), movies_to_consider, func)

# Display results
for name, recs in recommendations.items():
    print(f"Recommendations using {name} similarity:")
    print(recs)
    print()

Recommendations using Manhattan similarity:
[(3.53, 'Movie_2'), (3.46, 'Movie_8'), (3.16, 'Movie_7'), (3.01, 'Movie_19'), (2.92, 'Movie_16'), (2.8, 'Movie_6'), (2.58, 'Movie_10'), (2.34, 'Movie_9')]

Recommendations using Euclidean similarity:
[(3.47, 'Movie_2'), (3.41, 'Movie_8'), (3.25, 'Movie_7'), (2.92, 'Movie_19'), (2.87, 'Movie_16'), (2.85, 'Movie_6'), (2.72, 'Movie_10'), (2.39, 'Movie_9')]

Recommendations using Pearson similarity:
[(3.56, 'Movie_8'), (3.28, 'Movie_7'), (3.05, 'Movie_9'), (2.95, 'Movie_10'), (2.94, 'Movie_2'), (2.92, 'Movie_6'), (2.58, 'Movie_19'), (2.44, 'Movie_16')]

Recommendations using Cosine similarity:
[(3.4, 'Movie_8'), (3.32, 'Movie_7'), (3.31, 'Movie_2'), (3.01, 'Movie_19'), (2.89, 'Movie_6'), (2.84, 'Movie_10'), (2.66, 'Movie_16'), (2.55, 'Movie_9')]

Recommendations using Chebyshev similarity:
[(3.43, 'Movie_8'), (3.42, 'Movie_2'), (3.26, 'Movie_7'), (2.89, 'Movie_6'), (2.86, 'Movie_19'), (2.85, 'Movie_16'), (2.81, 'Movie_10'), (2.49, 'Movie_9')]

Re

## Question 05

In [81]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cityblock, euclidean, chebyshev, minkowski
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity

In [82]:
# Setting up parameters
np.random.seed(42)
num_critics = 50
num_movies = 250
missing_ratio = 0.4

# Generate ratings with random values and allow NaN for missing data
ratings = np.random.choice([1, 2, 3, 4, 5], size=(num_critics, num_movies)).astype(float)

In [83]:
# Function to apply missing data
def apply_missing_data(matrix, missing_ratio):
    total_cells = matrix.size
    num_missing = int(total_cells * missing_ratio)
    indices = [(i, j) for i in range(matrix.shape[0]) for j in range(matrix.shape[1])]
    missing_indices = random.sample(indices, num_missing)
    for i, j in missing_indices:
        matrix[i, j] = np.nan
    return matrix

ratings = apply_missing_data(ratings.copy(), missing_ratio)
critics = [f"Critic_{i+1}" for i in range(num_critics)]
movies = [f"Movie_{j+1}" for j in range(num_movies)]
ratings_df = pd.DataFrame(ratings, index=critics, columns=movies)

# Ensure target critic (Critic_1) has rated less than half the movies
ratings_df.iloc[0, :int(num_movies / 2)] = np.nan

In [84]:
# Updated similarity functions that handle missing data
def pearson_similarity_safe(p1, p2):
    mask = (~np.isnan(p1)) & (~np.isnan(p2))
    if mask.sum() < 2:  # At least two ratings needed for Pearson
        return 0
    return pearsonr(p1[mask], p2[mask])[0]

def cosine_similarity_safe(p1, p2):
    mask = (~np.isnan(p1)) & (~np.isnan(p2))
    if mask.sum() == 0:
        return 0
    return cosine_similarity([p1[mask]], [p2[mask]])[0][0]

def manhattan_similarity(p1, p2):
    mask = (~np.isnan(p1)) & (~np.isnan(p2))
    return -cityblock(p1[mask], p2[mask]) if mask.sum() else 0

def euclidean_similarity(p1, p2):
    mask = (~np.isnan(p1)) & (~np.isnan(p2))
    return -euclidean(p1[mask], p2[mask]) if mask.sum() else 0

def chebyshev_similarity_safe(p1, p2):
    mask = (~np.isnan(p1)) & (~np.isnan(p2))
    return -chebyshev(p1[mask], p2[mask]) if mask.sum() else 0

def minkowski_similarity_safe(p1, p2, p=3):
    mask = (~np.isnan(p1)) & (~np.isnan(p2))
    return -minkowski(p1[mask], p2[mask], p) if mask.sum() else 0

# Assigning similarity measures to dictionary
similarity_measures_safe = {
    "Pearson": pearson_similarity_safe,
    "Cosine": cosine_similarity_safe,
    "Manhattan": manhattan_similarity,
    "Euclidean": euclidean_similarity,
    "Chebyshev": chebyshev_similarity_safe,
    "Minkowski": lambda p1, p2: minkowski_similarity_safe(p1, p2, p=3)
}

# Placeholder for recommendations based on each similarity measure
recommendations_safe = {}
critiques = ratings_df.to_dict(orient="index")


In [85]:
# Target critic for recommendations
target_critic = 'Critic_1'
movies_to_consider = ratings_df.columns.tolist()

# Dummy function to simulate recommendation calculation
def calculate_recommendations(target, critiques, movies, similarity_func):
    scores = {}
    for critic, ratings in critiques.items():
        if critic != target:
            p1 = np.array([ratings[movie] for movie in movies])
            p2 = np.array([critiques[target][movie] for movie in movies])
            score = similarity_func(p1, p2)
            scores[critic] = score
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:3]

# Calculate recommendations for each similarity measure
for name, func in similarity_measures_safe.items():
    recommendations_safe[name] = calculate_recommendations(target_critic, critiques, movies_to_consider, func)

# Filter recommendations to find cases with completely different recommendations
unique_recommendations = {}
for name, recs in recommendations_safe.items():
    unique_recommendations[name] = {rec[0] for rec in recs}  # Store only critic names for comparison

# Check for complete difference in top recommendations
if len(set.intersection(*unique_recommendations.values())) == 0:
    # Print recommendations only if they are completely different
    for name, recs in recommendations_safe.items():
        print(f"Recommendations using {name} similarity:")
        for i, recommendation in enumerate(recs, start=1):
            print(f"{i}. {recommendation}")
        print()
else:
    print("No completely different recommendations across all six similarity measures.")


Recommendations using Pearson similarity:
1. ('Critic_3', 0.30327308025146926)
2. ('Critic_35', 0.29331900317538795)
3. ('Critic_26', 0.22835899043463817)

Recommendations using Cosine similarity:
1. ('Critic_35', 0.8795989942670849)
2. ('Critic_26', 0.8620508964742682)
3. ('Critic_28', 0.861475859139698)

Recommendations using Manhattan similarity:
1. ('Critic_6', -49.0)
2. ('Critic_36', -57.0)
3. ('Critic_3', -58.0)

Recommendations using Euclidean similarity:
1. ('Critic_6', -10.535653752852738)
2. ('Critic_24', -11.401754250991377)
3. ('Critic_35', -11.532562594670797)

Recommendations using Chebyshev similarity:
1. ('Critic_22', -3.0)
2. ('Critic_24', -3.0)
3. ('Critic_35', -3.0)

Recommendations using Minkowski similarity:
1. ('Critic_24', -6.5731384514424285)
2. ('Critic_6', -6.701759395378069)
3. ('Critic_35', -6.760614301748689)

